In [1]:
# import libraries
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import safe as sf
import datamol as dm

# Load pre-trained model
designer = sf.SAFEDesign.load_default(verbose=True)
designer.model

/home/affiah/anaconda3/envs/safe/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SAFEDoubleHeadsModel(
  (transformer): GPT2Model(
    (wte): Embedding(1880, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=1880, bias=False)
  (multiple_choice_head): PropertyHead(
    (summary): Linear(in_features=768, out_features=64, bias=True)
    

In [ ]:
### Scaffold Morphing
side_chains = "[1*]C(=O)C#CCN1CCCCC1.[2*]c1cccc(Br)c1"
dm.to_image(side_chains)

In [ ]:
generated_smiles = designer.scaffold_morphing(
    side_chains=side_chains,
    n_samples_per_trial=12,
    n_trials=1,
    sanitize=True,
    do_not_fragment_further=False,
    random_seed=100,
)

generated_smiles

In [ ]:
dm.to_image(generated_smiles[:12], mol_size=(350, 200))

## Testing core structure

In [3]:
# Define the core structure of triazolopyrazine
core_structure = "N1=CC=N[N]1"

# Generate scaffold morphing
generated_smiles = designer.scaffold_morphing(
    core=core_structure,
    n_samples_per_trial=12,
    n_trials=1,
    sanitize=True,
    do_not_fragment_further=False,
    random_seed=100,
)

# Display the generated SMILES
print(generated_smiles)

ArgumentError: Python argument types in
    rdkit.Chem.rdmolops.ReplaceCore(NoneType, Mol)
did not match C++ signature:
    ReplaceCore(RDKit::ROMol mol, RDKit::ROMol coreQuery, bool replaceDummies=True, bool labelByIndex=False, bool requireDummyMatch=False, bool useChirality=False)
    ReplaceCore(RDKit::ROMol mol, RDKit::ROMol core, boost::python::api::object matches, bool replaceDummies=True, bool labelByIndex=False, bool requireDummyMatch=False)

### Convert Molecule to Side Chain

In [ ]:
mol = "CC1=C(C(=NO1)C2=CC=CC=C2Cl)C(=O)NC3C4N(C3=O)C(C(S4)(C)C)C(=O)O"
core0 = "CC1(C)CN2C(CC2=O)S1"
core1 = "CC1(C)SC2C(-*)C(=O)N2C1-*"
core2 = "CC1N2C(SC1(C)C)C(N)C2=O"
side_chain = compute_side_chain(core=core0, mol=mol)
dm.to_image([side_chain, core0, mol])

## Testing the main.py script

In [3]:
# imports
import os
import csv
# from rdkit import Chem
# from rdkit.Chem.Descriptors import MolWt
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import safe as sf


# convert SMILES to SAFE
def smiles_to_safe(smiles):
    try:
        safe = sf.encode(smiles)
        return safe
    # except sf.EncoderError:
    except:
        print("Error in SMILES conversion")


#  convert core structure to side chain
def side_chains(core_structure):
    return core_structure


# generate new smiles
def generate_smiles(side_chains):
    generated_smiles = designer.scaffold_morphing(
        side_chains=side_chains,
        n_samples_per_trial=12,
        n_trials=1,
        sanitize=True,
        do_not_fragment_further=False,
        random_seed=100,)
    return generated_smiles

# my model
def my_model(side_chains_list):
    return [generate_smiles(side_chains) for side_chains in side_chains_list]


# read SMILES from .csv file, assuming one column with header
with open("data/my_molecules.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    smiles_list = [r[0] for r in reader]

# Load pre-trained model
designer = sf.SAFEDesign.load_default(verbose=True)
designer.model

# convert to SAFE
safe_list = [smiles_to_safe(smi) for smi in smiles_list]

# convert molecule to side chains
side_chains_list = [side_chains(sf) for sf in safe_list]

# run model
outputs = my_model(smiles_list)
outputs

2024-01-17 14:42:37.121 | WARNING  | safe.sample:_fragment_linking:239 - Side chain NCCS does not contain any dummy atoms, this might not be what you want


Error in SMILES conversion


IndexError: list index out of range